# Brain Tumor Classification using CNN
**Author:** Samay Asubadin  
**Degree:** Biomedical Engineering, Korea University  

This notebook presents the exploratory analysis, visualization, and qualitative evaluation
of a Convolutional Neural Network trained to classify brain MRI images into four categories:
glioma, meningioma, pituitary tumor, and non-tumor.

The core training and evaluation pipeline is implemented in `Brain_tumor_SamayAsubadin.py`.


In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix


ImportError: Traceback (most recent call last):
  File "C:\Users\samya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

Initialize the environment and import libraries used exclusively for analysis and visualization.

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/Brain_tumor_dataset"  # Colab runtime path
TRAIN_DIR = os.path.join(DATASET_DIR, "Train")
TEST_DIR  = os.path.join(DATASET_DIR, "Test")

classes = ['glioma', 'meningioma', 'nontumor', 'pituitary']


In [ ]:
for cls in classes:
    path = os.path.join(TRAIN_DIR, cls)
    print(f"{cls}: {len(os.listdir(path))} images")


## Class Distribution Visualization

In [ ]:
class_counts = [len(os.listdir(os.path.join(TRAIN_DIR, c))) for c in classes]

plt.figure(figsize=(8, 5))
sns.barplot(x=classes, y=class_counts)
plt.title("Training Set Class Distribution")
plt.ylabel("Number of Images")
plt.show()


## Visualizing Random Training Samples
Ransom samples are visualized to qualitatively inspect image quality, contrast, and inter-class variability.

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(15, 10))
fig.suptitle("Random Training Samples", fontsize=18)

for ax in axes.flatten():
    cls = random.choice(classes)
    img_name = random.choice(os.listdir(os.path.join(TRAIN_DIR, cls)))
    img_path = os.path.join(TRAIN_DIR, cls, img_name)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))

    ax.imshow(img, cmap='gray')
    ax.set_title(cls)
    ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Load Train model
MODEL_PATH = "/content/drive/MyDrive/brain_tumor_project/models/brain_tumor_cnn_model.keras"
model = load_model(MODEL_PATH)


## Confusion Matrix

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=1,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)
y_true = test_gen.classes


In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=classes,
            yticklabels=classes,
            cmap="Blues")

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


## Predicted vs. Actual Image Classification
This visualization highlights both correct and incorrect predictions,
providing insight into common failure modes and class-specific ambiguity.

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(15, 10))
fig.suptitle("Predicted vs Actual Labels", fontsize=18)

for i, ax in enumerate(axes.flatten()):
    img_path = test_gen.filepaths[i]
    true_label = classes[y_true[i]]
    pred_label = classes[y_pred[i]]

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))

    color = "green" if true_label == pred_label else "red"

    ax.imshow(img, cmap="gray")
    ax.set_title(f"Pred: {pred_label}\nActual: {true_label}", color=color)
    ax.axis("off")

plt.tight_layout()
plt.show()


## Training Curves

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title("Loss")
plt.legend()
plt.show()


## Interpretation 
The model demonstrates strong performance across tumor classes. Misclassification primarily occurs between visually similar tumor types, suggesting potential benefit from higher-resolution inputs or attention mechanisms.